## 웹크롤링

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
interval = 0.5
# interval = 1 # <- headlless 크롤링

url = "https://sgis.kostat.go.kr/view/catchmentArea/main#"

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")

# 크롬창을 띄우지 않고 실행(headless)
options.headless = True
# 윈도우창 사이즈 설정
options.add_argument("window-size=2560x1600")

browser = webdriver.Chrome(options=options)
browser.get(url)

# 윈도우창 최대화
browser.maximize_window()

In [2]:
# 팝업창 "취소" 클릭
cancel = browser.find_element_by_class_name("ques02")
cancel.click()

In [3]:
# 웹페이지 정보 가져오기
soup = BeautifulSoup(browser.page_source, "lxml")

In [4]:
# 행정구역 "도" 클릭
provinces = browser.find_element_by_class_name("selct_91")
provinces.click()
time.sleep(interval)

In [5]:
# 서울특별시 선택
seoul = browser.find_element_by_css_selector("#sido > option:nth-child(1)")
seoul.click()
time.sleep(interval)

In [6]:
# 행정구역 "구" 클릭
districts = browser.find_element_by_class_name("selct_92")
districts.click()
time.sleep(interval)

In [4]:

# 데이터가 필요한 구 추가 및 제거
districts_list = ['강남구','강동구','서초구','송파구','강북구','광진구','노원구','도봉구','동대문구',
                  '성동구','성북구','중랑구','강서구','관악구','구로구','금천구','동작구','양천구',
                  '영등포구','마포구','서대문구','용산구','은평구','종로구','중구'
                 ]
districts_list.sort()

# 데이터가 필요한 편의시설 추가 및 제거
categories = ["지하철", "은행", "대형마트", "전통시장", "병원", "응급의료기관",
              "공원", "스포츠센터", "보건소", "사회복지시설", "행정기관", "백화점"
             ] 

In [8]:
# 웹 크롤링 및 엑셀파일 만들기
import pandas as pd

# 엑셀파일 이름 설정 및 생성
writer = pd.ExcelWriter('편의시설 데이터.xlsx', engine="openpyxl") 

for district in districts_list:
    select_district = browser.find_element_by_xpath(f"//option[contains(text(),'{district}')]")
    select_district.click()
    time.sleep(interval)
    print(f'{district} 시작합니다.') 
    
    # 빈 리스트 생성
    result = []  
    for category in categories:
        
        # 편의시설 분류 창 스크롤을 내리기
        time.sleep(interval)        
        small_window = browser.find_element_by_id("mCSB_1")
        browser.execute_script("arguments[0].scrollBy(0,400)", small_window)

        
        # 리스트 내 카테고리 선택 및 실행
        select_category = browser.find_element_by_xpath(f"//a[contains(text(),'{category}')]")
        browser.execute_script("arguments[0].click();", select_category)
        time.sleep(interval)
        soup = BeautifulSoup(browser.page_source, "lxml")
        category_data = soup.select_one("ul.res_li") 
        
        # print(category) # 카테고리 선택이 잘 되었는지 확인

        while category_data:
            for item in category_data :

                # 편의시설 별 데이터 추출
                name = item.select('li > span[class="sa_txt01"]')[0].get_text()
                category_name = item.select('li > span[class="txt02"]')
                location = item.select('li > div[class="txt03"]')

                if category_name and location != None:
                    category_name = item.select('li > span[class="txt02"]')[0].get_text()
                    location = item.select('li > div[class="txt03"]')[0].get_text()
                else :
                    category_name = soup.find("span", attrs={"id":"ftsdText"}).get_text()
                    location = '자료가 없습니다.'

                data = [district, name, category_name, location]
                # print(data) # 각 시설 별 데이터가 잘 출력되고 있는지 확인
                result.append(data)
            break

        # 데이터프레임으로 입력 및 엑셀로 추출
        df = pd.DataFrame(result)
        df.columns = ["district", "name", "category_name", "location"]
        df.to_excel(writer, index=False, sheet_name=district)

        # 카테고리로 돌아가기 위해 뒤로가기 버튼 실행
        backToCategory = browser.find_element_by_id("mapLocation_2")
        browser.execute_script("arguments[0].click();", backToCategory)
#         time.sleep(interval)
    print(f'{district} 완료합니다.', "\n")

print("웹 크롤링을 종료합니다.")
writer.save()
print("엑셀에 저장을 완료했습니다.")

강남구 시작합니다.
강남구 완료합니다. 

강동구 시작합니다.
강동구 완료합니다. 

강북구 시작합니다.
강북구 완료합니다. 

강서구 시작합니다.
강서구 완료합니다. 

관악구 시작합니다.
관악구 완료합니다. 

광진구 시작합니다.
광진구 완료합니다. 

구로구 시작합니다.
구로구 완료합니다. 

금천구 시작합니다.
금천구 완료합니다. 

노원구 시작합니다.
노원구 완료합니다. 

도봉구 시작합니다.
도봉구 완료합니다. 

동대문구 시작합니다.
동대문구 완료합니다. 

동작구 시작합니다.
동작구 완료합니다. 

마포구 시작합니다.
마포구 완료합니다. 

서대문구 시작합니다.
서대문구 완료합니다. 

서초구 시작합니다.
서초구 완료합니다. 

성동구 시작합니다.
성동구 완료합니다. 

성북구 시작합니다.
성북구 완료합니다. 

송파구 시작합니다.
송파구 완료합니다. 

양천구 시작합니다.
양천구 완료합니다. 

영등포구 시작합니다.
영등포구 완료합니다. 

용산구 시작합니다.
용산구 완료합니다. 

은평구 시작합니다.
은평구 완료합니다. 

종로구 시작합니다.
종로구 완료합니다. 

중구 시작합니다.
중구 완료합니다. 

중랑구 시작합니다.
중랑구 완료합니다. 

웹 크롤링을 종료합니다.
엑셀에 저장을 완료했습니다.


In [9]:
# 웹드라이버 종료
print("웹드라이버를 종료합니다.")
quit()

웹드라이버를 종료합니다.


## 데이터 전처리

In [ ]:
# (1) 사회복지시설 : '아동', '다문화', '직장', '장애인', '청소년', '육아'  키워드 제외 
# (2) 공원 데이터는 서울시 공원통계로 대체 
# (3) 스포츠센터 : 사무소 및 비관련 분야 제외 

# 전처리를 위한 키워드 추가 및 제거
keywords = ['장애인','육아','청소년',
            '아동', '직장','다문화',
            '현재 시군구에는 해당 시설 유형이 존재하지 않습니다.']

In [20]:
# 전처리된 데이터를 저장하기 위해 엑셀파일 만들기
writer = pd.ExcelWriter('편의시설 데이터(전처리 후).xlsx', engine='openpyxl')

# 빈 데이터프레임을 생성
df_count = pd.DataFrame(columns=categories)

for district in districts_list:
    df_district = pd.read_excel('편의시설 데이터.xlsx', sheet_name=district) # raw 데이터 파일 불러오기 
    print(f'필터링 전 {district} : {df_district["name"].count()}개')
    
    filterings = df_district[df_district['name'].str.contains('|'.join(keywords))]
    df_district.drop(filterings.index, inplace=True)    
    print(f'필터링 후 {district} : {df_district["name"].count()}개', "\n")
        
    # 필터링 된 데이터를 새로운 액셀파일에 저장
    df_district.to_excel(writer, index=False, sheet_name=district)
    
    
    # 편의시설 개수를 빈 데이터프레임에 추가  
    df_district.drop(columns=['name','location'], inplace=True)
    df_temp = df_district.groupby(by='category_name').count().T
    df_temp.rename(index={'district':district}, inplace=True)
    df_count = df_count.append(df_temp)

# 결측치 처리 후 csv 파일로 저장
df_count.fillna(0, inplace=True)
print("********** 각 자치구의 편의시설 개수 **********", "\n")
print(df_count)
# df_count.to_excel(writer, index=True, sheet_name='total district')
df_count.to_csv('편의시설 데이터(전처리 후).csv', index=True, index_label='구역')
writer.save()

필터링 전 강남구 : 617개
필터링 후 강남구 : 597개 

필터링 전 강동구 : 537개
필터링 후 강동구 : 511개 

필터링 전 강북구 : 346개
필터링 후 강북구 : 313개 

필터링 전 강서구 : 309개
필터링 후 강서구 : 274개 

필터링 전 관악구 : 185개
필터링 후 관악구 : 155개 

필터링 전 광진구 : 442개
필터링 후 광진구 : 422개 

필터링 전 구로구 : 537개
필터링 후 구로구 : 507개 

필터링 전 금천구 : 349개
필터링 후 금천구 : 314개 

필터링 전 노원구 : 230개
필터링 후 노원구 : 202개 

필터링 전 도봉구 : 354개
필터링 후 도봉구 : 325개 

필터링 전 동대문구 : 443개
필터링 후 동대문구 : 422개 

필터링 전 동작구 : 459개
필터링 후 동작구 : 426개 

필터링 전 마포구 : 640개
필터링 후 마포구 : 606개 

필터링 전 서대문구 : 375개
필터링 후 서대문구 : 350개 

필터링 전 서초구 : 413개
필터링 후 서초구 : 395개 

필터링 전 성동구 : 183개
필터링 후 성동구 : 160개 

필터링 전 성북구 : 471개
필터링 후 성북구 : 440개 

필터링 전 송파구 : 377개
필터링 후 송파구 : 345개 

필터링 전 양천구 : 581개
필터링 후 양천구 : 545개 

필터링 전 영등포구 : 724개
필터링 후 영등포구 : 698개 

필터링 전 용산구 : 346개
필터링 후 용산구 : 333개 

필터링 전 은평구 : 91개
필터링 후 은평구 : 91개 

필터링 전 종로구 : 308개
필터링 후 종로구 : 280개 

필터링 전 중구 : 580개
필터링 후 중구 : 565개 

필터링 전 중랑구 : 201개
필터링 후 중랑구 : 161개 

********** 각 자치구의 편의시설 개수 ********** 

      지하철   은행  대형마트  전통시장  병원  응급의료기관  공원  스포츠센터  보건소  사회복

## 시각화

필요한 열 : 유통점, 총합계, 65세이상 비중, 공원

In [38]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [39]:
# csv 파일 불러오기
df=pd.read_csv('편의시설 데이터(전처리 후).csv', index_col='구역')
df.head(3)

,지하철,은행,대형마트,전통시장,병원,응급의료기관,공원,스포츠센터,보건소,사회복지시설,행정기관,백화점
구역,,,,,,,,,,,,
강남구,26,328,3,5,62,2,1,0,1,143,22,4
강동구,10,60,3,10,28,3,3,332,2,41,18,1
강북구,4,25,1,11,17,0,3,193,3,39,16,1


In [32]:
#폰트 설정
plt.rc('font', family='Malgun Gothic')

#컬러맵
from matplotlib import cm

cmaps = plt.colormaps()
for cm in cmaps:
    print(cm)

Accent
Accent_r
Blues
Blues_r
BrBG
BrBG_r
BuGn
BuGn_r
BuPu
BuPu_r
CMRmap
CMRmap_r
Dark2
Dark2_r
GnBu
GnBu_r
Greens
Greens_r
Greys
Greys_r
OrRd
OrRd_r
Oranges
Oranges_r
PRGn
PRGn_r
Paired
Paired_r
Pastel1
Pastel1_r
Pastel2
Pastel2_r
PiYG
PiYG_r
PuBu
PuBuGn
PuBuGn_r
PuBu_r
PuOr
PuOr_r
PuRd
PuRd_r
Purples
Purples_r
RdBu
RdBu_r
RdGy
RdGy_r
RdPu
RdPu_r
RdYlBu
RdYlBu_r
RdYlGn
RdYlGn_r
Reds
Reds_r
Set1
Set1_r
Set2
Set2_r
Set3
Set3_r
Spectral
Spectral_r
Wistia
Wistia_r
YlGn
YlGnBu
YlGnBu_r
YlGn_r
YlOrBr
YlOrBr_r
YlOrRd
YlOrRd_r
afmhot
afmhot_r
autumn
autumn_r
binary
binary_r
bone
bone_r
brg
brg_r
bwr
bwr_r
cividis
cividis_r
cool
cool_r
coolwarm
coolwarm_r
copper
copper_r
crest
crest_r
cubehelix
cubehelix_r
flag
flag_r
flare
flare_r
gist_earth
gist_earth_r
gist_gray
gist_gray_r
gist_heat
gist_heat_r
gist_ncar
gist_ncar_r
gist_rainbow
gist_rainbow_r
gist_stern
gist_stern_r
gist_yarg
gist_yarg_r
gnuplot
gnuplot2
gnuplot2_r
gnuplot_r
gray
gray_r
hot
hot_r
hsv
hsv_r
icefire
icefire_r
inferno
infern

In [35]:
park = np.array(df['공원'])[:]
health_center = np.array(df['보건소'])[:]
hospital = np.array(df['병원'])[:]
welfare_organization = np.array(df['사회복지시설'])[:]
sports_center = np.array(df['스포츠센터'])[:]
bank = np.array(df['은행'])[:]
casualty = np.array(df['응급의료기관'])[:]
market = np.array(df['전통시장'])[:]
subway = np.array(df['지하철'])[:]
government_agency = np.array(df['행정기관'])[:]